<a href="https://colab.research.google.com/github/saridormi/notebooks/blob/master/BERT_for_Source_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT for Source Code

### Александра Елисеева, CSC

Попробуем дообучить [CodeBERT](https://github.com/microsoft/CodeBERT) для задачи Function-Docstring Mismatch.

## Скачиваем данные и создаем датасет для Function-Docstring Mismatch

Установим необходимые зависимости.

In [ ]:
!pip install torch==1.4.0 torchvision==0.5.0 # torchvision 0.7.0 requires torch 1.6.0 and produces an error
# transformers 2.5.0 was stated in CodeBERT repo but I really liked 3.1.0 docs 
# and docs from 2.5.0 very MUCH less informative so I decided to experiment
# with newer version first
!pip install transformers==3.1.0
!pip install filelock more_itertools

Насколько я понимаю, на вход модели CodeBERT подается токенизированный код. Поэтому нам нужны не распарсенные AST, а сами исходники. 

Для удобства скачаем архив с исходными файлами датасета [Py150](https://www.sri.inf.ethz.ch/py150) на мой гугл диск, чтобы выполнять только часть действий при каждом новом подключении к Colab. 

In [ ]:
from google.colab import drive
import os
drive.mount('gdrive', force_remount=True)
drive_path = "gdrive/My Drive/"

In [ ]:
if 'py150' in os.listdir():
  print("Dataset already downloaded and extracted")
else:
  if 'py150_files.tar.gz' not in os.listdir(drive_path):
    print("Downloading dataset")
    !wget http://files.srl.inf.ethz.ch/data/py150_files.tar.gz -P "gdrive/My Drive"
    print("Dataset downloaded!")
  print("Extracting dataset")
  !mkdir py150
  !tar xzvf "gdrive/My Drive/py150_files.tar.gz" -C py150
  print("Extraction done!")
if "data" not in os.listdir("py150"):
  print("Extracting source code")
  !tar xzvf "py150/data.tar.gz" -C py150  
  print("Extraction done!")
else:
  print("Source code already extracted")  

Теперь воспользуемся модулем [ast](https://docs.python.org/3.8/library/ast.html), чтобы выделить из кода функции и docstring-и к ним. При этом в google colab используется Python версии 3.6.9, и воспользоваться прекрасной функцией [get_source_segment](https://docs.python.org/3.8/library/ast.html#ast.get_source_segment) не получается, поэтому я решила обратиться к некоторой сторонней библиотеке [astunparse](https://astunparse.readthedocs.io/en/latest/).

In [3]:
!python -V

Python 3.6.9


In [ ]:
!pip install astunparse==1.5.0

Напишем необходимый класс и функции и поэкспериментируем на одном случайном файле:

In [5]:
import ast
import astunparse
from pprint import pprint


def format_str(string):
  """Removes all possible newline characters from string."""
  for char in ['\r\n', '\r', '\n']:
    string = string.replace(char, ' ')
  return string


class FunctionVisitor(ast.NodeVisitor):
    def __init__(self):
      """Create empty list to store strings of format docstring<CODESPLIT>function."""
      self.stats = []
      
    def visit_FunctionDef(self, node):
      """When visiting FunctionDef node, saves doctring and function sourse
      code to self.stats. Functions without doctrings are discarded."""
      docstring = ast.get_docstring(node)
      if docstring is not None:
        node.body = node.body[1:] # remove docstring from function itself
        example = (format_str(docstring), format_str(astunparse.unparse(node)))
        example = '<CODESPLIT>'.join(example)
        self.stats.append(example)
      self.generic_visit(node)

    def report(self):
      """Print elements of self.stats"""
      for el in self.stats:
        print(el)

    def write_to_file(self, filename, mode='w'):
      """Write all strings of format docstring<CODESPLIT>function to filename
      with given mode ('w'/'a'). Each string is written on new line."""
      with open(filename, mode, encoding='utf-8') as f:
        f.writelines('\n'.join(self.stats))


def process_file(infilename, outfilename, mode='w'):
  """Open file, located at path infilename, process each function with docstring
  into format docstring<CODESPLIT>function and write it into outfilename."""
  with open(infilename, encoding='utf-8') as ex:
    try:
      tree = ast.parse(ex.read())
    except SyntaxError:
      print(f"Syntax error while parsing {infilename}")
      return
    except UnicodeDecodeError:  
      print(f"Unicode decode error while parsing {infilename}")
      return
    vis = FunctionVisitor()
    vis.visit(tree)
    vis.write_to_file(outfilename, mode)

example_file = "py150/data/Blizzard/heroprotocol/protocol29406.py"
process_file(example_file, 'out.txt')
with open('out.txt') as f:
  pprint(f.readlines())

['Decodes and yields each game event from the contents byte '
 'string.<CODESPLIT>  def decode_replay_game_events(contents):     decoder = '
 'BitPackedDecoder(contents, typeinfos)     for event in '
 '_decode_event_stream(decoder, game_eventid_typeid, game_event_types, '
 'decode_user_id=True):         (yield event) \n',
 'Decodes and yields each message event from the contents byte '
 'string.<CODESPLIT>  def decode_replay_message_events(contents):     decoder '
 '= BitPackedDecoder(contents, typeinfos)     for event in '
 '_decode_event_stream(decoder, message_eventid_typeid, message_event_types, '
 'decode_user_id=True):         (yield event) \n',
 'Decodes and yields each tracker event from the contents byte '
 'string.<CODESPLIT>  def decode_replay_tracker_events(contents):     decoder '
 '= VersionedDecoder(contents, typeinfos)     for event in '
 '_decode_event_stream(decoder, tracker_eventid_typeid, tracker_event_types, '
 'decode_user_id=False):         (yield event) \n',
 'D

В принципе, получилось то, что нужно, поэтому воспользуемся этой функцией для получения данных positive класса:

In [6]:
def create_positive_dataset(mode):
  """Using .txt files, provided in Py150 dataset, create file with lines of 
  format docstring<CODESPLIT>function from all specified source code."""
  if mode not in os.listdir():
    !mkdir "{mode}"
  !rm -rf "{mode}"/positive.txt
  if mode == 'train':
    infile = "py150/python100k_train.txt"
  else:
    infile = "py150/python50k_eval.txt"
  with open(infile, encoding='utf-8') as f:
    for filename in f:
      try:
        process_file(os.path.join('py150', filename.strip()), f'{mode}/positive.txt', 'a')
      except:
        pass

In [ ]:
create_positive_dataset('train')
create_positive_dataset('test')

Во время построения AST возникло много исключений вида `SyntaxError` и `UnicodeDecodeError`, так что, возможно, я что-то делаю не так. Посмотрим, сколько у нас получилось примеров для train и test и подумаем, насколько это существенная проблема на данном этапе:

In [8]:
ds_len = {}
for mode in ['train', 'test']:
  with open(f"{mode}/positive.txt") as f:
    for i, l in enumerate(f):
      pass
  ds_len[mode] = i + 1
  print(f"Positive {mode} examples:", i + 1)

Positive train examples: 163472
Positive test examples: 80122


Будем считать, что этого достаточно. Как вариант для исправления этой проблемы, стоит посмотреть на файлы, выбрасывающие эти исключения, а еще - на файл из распарсенного варианта Py150, парсящий код.

Итак, на настоящий момент у нас есть данные positive класса. Следуя подходу из статьи [CuBERT](https://arxiv.org/pdf/2001.00059.pdf), для получения negative класса сделаем permutation всех строчек. 

In [9]:
import numpy as np


def create_negative_dataset(mode):
  """Randomly shuffle docstrings & functions from positive examples."""
  !rm -rf "{mode}"/negative.txt
  
  index = np.random.permutation(ds_len[mode])

  with open(f'{mode}/positive.txt') as infile:
    lines = [line.rstrip('\n') for line in infile]
  with open(f'{mode}/negative.txt', 'w') as outfile:
    for i, j in enumerate(index):
      outfile.write(f'{lines[i].split("<CODESPLIT>")[0]}<CODESPLIT>{lines[j].split("<CODESPLIT>")[1]}\n')

In [10]:
create_negative_dataset('train')
create_negative_dataset('test')

Опять же, сохраним архивы из train и test на гугл диск, чтобы не запускать это заново каждый раз.

In [11]:
!tar -zcvf "gdrive/My Drive/train.tar.gz" train
!tar -zcvf "gdrive/My Drive/test.tar.gz" test

train/
train/negative.txt
train/positive.txt
test/
test/negative.txt
test/positive.txt


In [8]:
!tar -xvzf "gdrive/My Drive/train.tar.gz" -C .
!tar -xvzf "gdrive/My Drive/test.tar.gz" -C .

train/
train/negative.txt
train/positive.txt
test/
test/negative.txt
test/positive.txt


И посмотрим на пример:

In [9]:
with open("train/positive.txt") as pos:
  print(pos.readline())
with open("train/negative.txt") as neg:
  print(neg.readline())

Add news items to the home page.<CODESPLIT>  @user_passes_test((lambda u: u.is_staff)) def add(request):     if (request.method == 'POST'):         form = AddItemForm(data=request.POST)         if form.is_valid():             item = form.save(commit=False)             item.reporter = request.user             try:                 with transaction.atomic():                     item.save()             except twitter.TwitterError as e:                 messages.error(request, ('Twitter error: "%s" Please try again.' % e.message[0]['message']))             else:                 messages.info(request, 'Your news item has been published!')                 return redirect('home')     else:         form = AddItemForm()     return render(request, 'form.html', {'title': 'Add Item', 'form': form, 'description': 'Enter the details for the news item below.', 'action': 'Add'})         <CODESPLIT>  def __init__(self, timeout=0):     self._hostname = ''     self._urls = []     self.connection = None 

A

Действительно, docstring-и одинаковые, а сам код функций отличается. Здесь можно заметить еще один баг - в первой строчке docstring один, а функции две :( 

## Приводим датасет к нужному формату

Теперь приведем это к виду, нужному для обучения модели, опираясь на [данный](https://huggingface.co/transformers/custom_datasets.html) туториал от HuggingFace.

Для начала, создадим отдельные списки с docstring-ами и самим кодом функций, а также лейблы к ним.

In [11]:
from sklearn.model_selection import train_test_split


def read_data(split_dir):
    """Save data from .txt files to docstrings and functions lists
    separately and label them to use in "sequence classification"-like 
    downstream task."""
    docs = []
    funcs = []
    labels = []
    for data_file in ["positive.txt", "negative.txt"]:
      with open(f"{split_dir}/{data_file}") as f:
        for line in f:
          doc, func = line.strip().split("<CODESPLIT>")[:2]
          docs.append(doc)
          funcs.append(func)
          labels.append(0 if data_file is "negative.txt" else 1)
    return docs, funcs, labels


train_docs, train_funcs, train_labels = read_data('train')
test_docs, test_funcs, test_labels = read_data('test')

Затем импортируем необходимые для обучения и работы с данными библиотеками и загрузим предобученную модель. В данном случае нам не нужно заниматься Mask Prediction, поэтому мы можем выбрать `codebert-base`, а не `codebert-base-mlm`.

Также я решила использовать `RobertaForSequenceClassification` вместо `RobertaModel`, потому что там, кроме основной модели, уже есть классификатор.

In [12]:
import torch
from torch.utils.data import DataLoader, RandomSampler
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, RobertaModel, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
config = RobertaConfig.from_pretrained("microsoft/codebert-base", output_hidden_states=True)
model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", config=config)
model.to(device)
model.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

Можно посмотреть на полную архитектуру сети, а еще обратить внимание на два момента:

* В `codebert-base` есть веса только для базовой модели, из-за чего появляется предупреждение:


> Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Это не проблема, потому что мы как раз собираемся ОБУЧИТЬ модель.

* Классификатор состоит из следующих слоев:

```
(classifier): RobertaClassificationHead(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=2, bias=True)
)
```

На выходе данные имеют размерность 2, что как раз подходит для текущей задачи.

Токенизируем наши данные. Как указано в [документации](https://huggingface.co/transformers/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.__call__), в transformers 3.1.0 при применении tokenizer к данным сразу возвращается BatchEncoding.

In [13]:
train_encodings = tokenizer(train_docs, train_funcs, truncation=True, padding=True)
test_encodings = tokenizer(test_docs, test_funcs, truncation=True, padding=True)

Затем напишем кастомный класс, наследующий от `torch.utils.data.Dataset`, чтобы пользоваться функционалом PyTorch для работы с датасетами. 

In [14]:
class py150Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = py150Dataset(train_encodings, train_labels)
test_dataset = py150Dataset(test_encodings, test_labels)

## Обучаем модель

Перейдем к обучению. В статье [CodeBERT](https://arxiv.org/pdf/2002.08155.pdf) авторы используют следующую конфигурацию:

```
In the fine-turning step, we set the learning rate as 1e-5, the
batch size as 64, the max sequence length as 200 and the
max fine-tuning epoch as 8. We use Adam to update the
parameters.
```

Попробуем сделать то же самое, кроме `batch size` *(потому что памяти в колабе не хватает)* и `max sequence length` *(я не совсем понимаю, где это менять)*.



In [15]:
import time
from tqdm import tqdm_notebook as tqdm

train_sampler = RandomSampler(train_dataset)
# with bigger batch sizes I get cuda memory error =(
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=1)

optim = AdamW(model.parameters(), lr=1e-5)
print("Number of batches:", len(train_loader))
model.train()
for epoch in range(3):
    if epoch != 0:
      end = time.time()
      print("Epoch took", end-start, "seconds")
    print("Epoch:", epoch)
    start = time.time()
    for i, batch in tqdm(enumerate(train_loader)):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

Number of batches: 326944
Epoch: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


KeyboardInterrupt: ignored

C `batch_size = 1` у нас получается 326944 батчей на каждую эпоху. Работает это очень медленно, и для полного обучения хотя бы одну эпоху потребуется как минимум 5 часов. Я решила оставить на ~30 минут.

За это время модель обучилась всего на 12120 батчах из 326944, причем меньше эпохи.

Посмотрим на работу модели на нескольких случайных примерах из тестового датасета.

In [ ]:
# I ran into some CUDA errors here which blocked the session from using gpu
# completely so I decided to switch to cpu for this simple evaluation :)
device='cpu'
model.to('cpu')

In [37]:
def eval_model(pos_line, neg_line, verbose=False):
  """Run model on pos_line'th positive and neg_line'th negative examples from 
  test and print classification results."""
  classes = ["negative", "positive"]

  with open("test/positive.txt") as pos:
    for i in range(pos_line):
      pos.readline()
    pos_seq = pos.readline()
  with open("test/negative.txt") as neg:
    for i in range(neg_line):
      neg.readline()
    neg_seq = neg.readline()

  pos_doc, pos_func = pos_seq.strip().split("<CODESPLIT>")[:2]
  neg_doc, neg_func = neg_seq.strip().split("<CODESPLIT>")[:2]

  print(f"Positive input ({pos_line}th example)")
  if verbose:    
    print(pos_doc)
    print(pos_func)
  print(f"Negative input ({neg_line}th example)")
  if verbose:
    print(neg_doc)
    print(neg_func)
  print()

  positive = tokenizer(pos_doc, pos_func, return_tensors="pt").to(device)
  negative = tokenizer(neg_doc, neg_func, return_tensors="pt").to(device)

  model.eval()

  positive_classification_logits = model(positive['input_ids'], attention_mask=positive['attention_mask'])[0]
  negative_classification_logits = model(negative['input_ids'], attention_mask=negative['attention_mask'])[0]

  positive_results = torch.softmax(positive_classification_logits, dim=1).tolist()[0]
  negative_results = torch.softmax(negative_classification_logits, dim=1).tolist()[0]

  # Should be positive
  print("Positive results:")
  print(positive_results)
  for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(positive_results[i] * 100))}%")
  print()

  # Should be negative
  print("Negative results:")
  print(negative_results)
  for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(negative_results[i] * 100))}%")

In [41]:
import numpy as np

n = 3

for i in range(n):
  pos_line = np.random.randint(0, 10)
  neg_line = np.random.randint(0, 10)
  eval_model(pos_line, neg_line, True)
  print()

Positive input (8th example)
Service Discovery: disco#items 
  @xmpp.iq(('{%s}query' % __disco_items_ns__)) def disco_items(self, iq):     target = iq.get('to')     if (target.find('@') < 0):         query = self.E.query({'xmlns': __disco_items_ns__})     else:         query = self.E.query({'xmlns': __disco_items_ns__})     return self.iq('result', iq, query, {'from': target}) List all the active connections  Args:     filter:         A general filter/query string that narrows the list of         resources returned by a multi-resource GET (read) request and         DELETE (delete) request. The default is no filter         (all resources are returned). The filter parameter specifies         a general filter/query string. This query string narrows the         selection of resources returned from a GET request that         returns a list of resources. The following example shows how to         retrieve only the first 10 connections:  Returns: all the connections, filtered or not.
Negative

Во всех случаях модель уверенно отличила правильное сочетание docstring-функция от неправильного. 

При этом периодически возникали проблемы с индексами: я думаю, что это может быть связано с незаконченным обучением. Поэтому я решила оставить "evaluation" в его текущем состоянии всего на 3 примерах и, тем более, не оценивать результаты модели на всей тестовой выборке с использованием распространенных метрик.